<a href="https://colab.research.google.com/github/Tiabet/Complete_story/blob/master/KoGPT_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

%%capture
!pip install transformers
!pip install accelerate
!pip install datasets
!pip install rouge-score
!pip install konlpy
!pip install bert_score

In [44]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, DataCollatorWithPadding
from datasets import Dataset, load_dataset, concatenate_datasets
from transformers import TrainingArguments
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from konlpy.tag import Okt
from bert_score import score as bert_score

In [54]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [23]:
 model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

# 모델 테스트

In [35]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the input data (input_ids) to the GPU
input_ids = input_ids.to(device)

# Move the model to the GPU (assuming your model is already initialized)
model.to(device)
gen_ids = model.generate(input_ids,
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)

gen_ids = gen_ids.to("cpu")
generated = tokenizer.decode(gen_ids[0])
generated

'근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.\n특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.\n또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.\n아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.\n운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.\n운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.\n운동을'

정상적으로 작동하는 것 확인

In [25]:
!nvidia-smi

Thu Sep  7 13:18:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 데이터셋 전처리

In [8]:
dataset = load_dataset('drive/MyDrive', data_files={
    'train': 'nikluge-sc-2023-train.jsonl',
    'validation': 'nikluge-sc-2023-dev.jsonl',
    'test': 'nikluge-sc-2023-test.jsonl'
})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
type(dataset['train'])

datasets.arrow_dataset.Dataset

In [10]:
sentence1 = []
sentence3 = []

for dictionary in dataset['train']['input']:
    sentence1.append(dictionary["sentence1"])
    sentence3.append(dictionary["sentence3"])

In [11]:
dataset['train'] = dataset['train'].add_column('sentence1', sentence1)
dataset['train'] = dataset['train'].add_column('sentence3', sentence3)

In [12]:
dataset['train']

Dataset({
    features: ['id', 'input', 'output', 'sentence1', 'sentence3'],
    num_rows: 120140
})

In [13]:
sentence1_val = []
sentence3_val = []

for dictionary in dataset['validation']['input']:
    sentence1_val.append(dictionary["sentence1"])
    sentence3_val.append(dictionary["sentence3"])

dataset['validation'] = dataset['validation'].add_column('sentence1', sentence1_val)
dataset['validation'] = dataset['validation'].add_column('sentence3', sentence3_val)

In [14]:
dataset['validation']

Dataset({
    features: ['id', 'input', 'output', 'sentence1', 'sentence3'],
    num_rows: 15017
})

In [15]:
sentence1_test = []
sentence3_test = []

for dictionary in dataset['test']['input']:
    sentence1_test.append(dictionary["sentence1"])
    sentence3_test.append(dictionary["sentence3"])

dataset['test'] = dataset['test'].add_column('sentence1', sentence1_test)
dataset['test'] = dataset['test'].add_column('sentence3', sentence3_test)

In [16]:
dataset['test']

Dataset({
    features: ['id', 'input', 'output', 'sentence1', 'sentence3'],
    num_rows: 15018
})

In [70]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence3"], example['output'],truncation=True, padding='max_length',max_length=128, return_tensors="pt")

In [71]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/120140 [00:00<?, ? examples/s]

Map:   0%|          | 0/15017 [00:00<?, ? examples/s]

Map:   0%|          | 0/15018 [00:00<?, ? examples/s]

In [77]:
training_args  = TrainingArguments(
    output_dir="content",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    save_steps=500,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2,
    seed=7,
)

training_args.max_seq_length = 32


In [73]:
tokenized_datasets = tokenized_datasets.remove_columns(dataset["train"].column_names)

In [74]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 평가 지표 작성 및 모델 파인튜닝

In [46]:
# Your custom evaluation function
def evaluate(reference_text, generated_text):
    # Calculate ROUGE-1 score
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1'])
    rouge_scores = rouge_scorer_obj.score(reference_text, generated_text)
    rouge_1_score = rouge_scores['rouge1'].fmeasure

    # Tokenize using Okt
    okt = Okt()
    reference_sentences = [okt.morphs(s) for s in reference_text]
    generated_sentence = okt.morphs(generated_text)

    smoother = SmoothingFunction()
    bleu_score = sentence_bleu(reference_sentences, generated_sentence, weights=(1.0, 0, 0, 0), smoothing_function=smoother.method1)

    # Calculate BERTScore
    _, _, bert_score_f1 = bert_score([generated_text], [reference_text])
    bert_score_f1 = bert_score_f1.mean().item()

    # Calculate the mean of the three scores
    mean_score = (rouge_1_score + bleu_score + bert_score_f1) / 3.0

    return mean_score

In [78]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = evaluate
)


In [81]:
trainer.train()

Step,Training Loss,Validation Loss


OutOfMemoryError: ignored

In [80]:
torch.cuda.empty_cache()